In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import plotly.graph_objects as go
import json
from plotly.utils import PlotlyJSONEncoder

workout_df = pd.read_csv('workouts.csv')
# workout_df.head() # Para visualização opcional

In [2]:
workout_df_columns_filtered = workout_df[['title','start_time', 'exercise_title', 'set_index', 'set_type', 'weight_kg', 'reps']]
workout_df_columns_filtered = workout_df_columns_filtered.rename(columns={'start_time': 'date'})

# Converte 'date' para datetime e depois para date (sem a hora)
# Usar errors='coerce' para transformar datas inválidas em NaT, que podem ser tratadas depois se necessário
workout_df_columns_filtered['date'] = pd.to_datetime(workout_df_columns_filtered['date'], format="%d %b %Y, %H:%M", errors='coerce')
workout_df_columns_filtered = workout_df_columns_filtered.dropna(subset=['date']) # Remove linhas onde a data não pôde ser parseada
workout_df_columns_filtered['date'] = workout_df_columns_filtered['date'].dt.date

# Filtrar para os últimos 90 dias (ou o período máximo que você quer pré-processar)
hoje = datetime.now().date()
limite_maximo_historico = hoje - timedelta(days=90) # Ajuste conforme necessário

workout_df_processed = workout_df_columns_filtered[workout_df_columns_filtered['date'] >= limite_maximo_historico].copy()
# workout_df_processed.head() # Para visualização opcional

In [3]:
def gerar_grafico_plotly_para_html(df, treino_nome, exercicio_nome, dias_periodo=30, serie_selecionada="Top Set (Failure)"):
    """
    Gera os dados e o layout de um gráfico Plotly para um treino, exercício, período e tipo de série específicos.
    Retorna um dicionário {'data': ..., 'layout': ...} ou None.
    """
    df_copy = df.copy()

    exercicio_df_original = df_copy[(df_copy['title'] == treino_nome) & (df_copy['exercise_title'] == exercicio_nome)]
    if exercicio_df_original.empty:
        return None

    exercicio_df = exercicio_df_original.copy()
    has_failure = not exercicio_df[exercicio_df['set_type'] == 'failure'].empty
    
    if exercicio_df['date'].empty:
        return None
    max_date = exercicio_df['date'].max()

    exercicio_df.loc[:, 'normal_order'] = pd.NA
    for dt_unique in exercicio_df['date'].unique():
        subset_indices = exercicio_df[
            (exercicio_df['date'] == dt_unique) & (exercicio_df['set_type'] == 'normal')
        ].sort_values('set_index').index
        for i, idx in enumerate(subset_indices, 1):
            exercicio_df.loc[idx, 'normal_order'] = i
    exercicio_df['normal_order'] = exercicio_df['normal_order'].astype('Int64')

    periodo_inicio = max_date - timedelta(days=dias_periodo)
    exercicio_df_periodo = exercicio_df[exercicio_df['date'] >= periodo_inicio].copy()

    if exercicio_df_periodo.empty:
        return None

    traces_data = []
    work_orders_in_period = sorted(exercicio_df_periodo[exercicio_df_periodo['normal_order'].notna()]['normal_order'].unique())

    if has_failure and (serie_selecionada == "Top Set (Failure)" or serie_selecionada == "Todas as Séries"):
        fdata = exercicio_df_periodo[exercicio_df_periodo['set_type'] == 'failure']
        if not fdata.empty:
            traces_data.append(go.Scatter(
                x=fdata['date'], y=fdata['weight_kg'],
                text=[f"{r} reps" for r in fdata['reps']],
                mode='lines+markers+text', textposition='top center',
                name='Top Set (Failure)'
            ))

    for order_val in work_orders_in_period:
        if serie_selecionada == f"Work Set {int(order_val)}" or serie_selecionada == "Todas as Séries":
            wdata = exercicio_df_periodo[exercicio_df_periodo['normal_order'] == order_val]
            if not wdata.empty:
                traces_data.append(go.Scatter(
                    x=wdata['date'], y=wdata['weight_kg'],
                    text=[f"{r} reps" for r in wdata['reps']],
                    mode='lines+markers+text', textposition='top center',
                    name=f'Work Set {int(order_val)}'
                ))
    
    if not traces_data:
        return None

    layout_title = f"Evolução: {exercicio_nome} ({treino_nome})"
    if serie_selecionada != "Todas as Séries":
        layout_title += f" - {serie_selecionada}"
    layout_title += f" - Últimos {dias_periodo} dias"

    layout_dict = go.Layout(
        title=layout_title,
        xaxis=dict(title='Data', tickformat='%d/%m/%Y'),
        yaxis=dict(title='Peso (kg)'),
        hovermode='closest',
        margin=dict(t=80, b=50, l=50, r=30) 
    ).to_plotly_json()

    return {'data': [trace.to_plotly_json() for trace in traces_data], 'layout': layout_dict}

In [4]:
def gerar_dados_para_js(df, rotinas_exercicios_map):
    graficos_data_para_js = {}
    periodos_dias_options = [30, 60, 90]
    
    for rotina, exercicios in rotinas_exercicios_map.items():
        for exercicio in exercicios:
            temp_exercicio_df = df[(df['title'] == rotina) & (df['exercise_title'] == exercicio)]
            if temp_exercicio_df.empty or temp_exercicio_df['date'].empty:
                continue

            series_disponiveis_para_este_exercicio = ["Todas as Séries"]
            if not temp_exercicio_df[temp_exercicio_df['set_type'] == 'failure'].empty:
                series_disponiveis_para_este_exercicio.append("Top Set (Failure)")
            
            # Certifique-se de que max_date_temp é calculado sobre um DataFrame não vazio
            max_date_temp = temp_exercicio_df['date'].max() 
            ninety_days_ago_temp = max_date_temp - timedelta(days=90)
            
            temp_exercicio_df_copy = temp_exercicio_df.copy()
            temp_exercicio_df_copy.loc[:, 'normal_order_temp'] = pd.NA
            for dt_unique_temp in temp_exercicio_df_copy['date'].unique():
                subset_indices_temp = temp_exercicio_df_copy[
                    (temp_exercicio_df_copy['date'] == dt_unique_temp) & 
                    (temp_exercicio_df_copy['set_type'] == 'normal')
                ].sort_values('set_index').index
                for i_temp, idx_temp in enumerate(subset_indices_temp, 1):
                    temp_exercicio_df_copy.loc[idx_temp, 'normal_order_temp'] = i_temp
            temp_exercicio_df_copy['normal_order_temp'] = temp_exercicio_df_copy['normal_order_temp'].astype('Int64')

            potential_work_orders = sorted(temp_exercicio_df_copy[
                (temp_exercicio_df_copy['normal_order_temp'].notna()) &
                (temp_exercicio_df_copy['date'] >= ninety_days_ago_temp) 
            ]['normal_order_temp'].unique())
            
            for ws_order in potential_work_orders:
                series_disponiveis_para_este_exercicio.append(f"Work Set {int(ws_order)}")

            for periodo_val in periodos_dias_options:
                for tipo_serie_val in series_disponiveis_para_este_exercicio:
                    chave_js = f"{rotina}||{exercicio}||{periodo_val}||{tipo_serie_val}"
                    grafico_spec = gerar_grafico_plotly_para_html(df, rotina, exercicio, 
                                                                  dias_periodo=periodo_val, 
                                                                  serie_selecionada=tipo_serie_val)
                    if grafico_spec:
                        graficos_data_para_js[chave_js] = grafico_spec
                        
    return graficos_data_para_js

In [5]:
# Criar mapa de rotinas para exercícios para o JavaScript
rotinas_exercicios_js_map = {}
treino_nomes_unicos = workout_df_processed['title'].unique()
for nome_treino_map in treino_nomes_unicos:
    exercicios_map_lista = workout_df_processed[workout_df_processed['title'] == nome_treino_map]['exercise_title'].unique().tolist()
    rotinas_exercicios_js_map[nome_treino_map] = sorted(exercicios_map_lista) # Ordenar para consistência

# Gerar o dicionário de especificações de gráficos para JS
graficos_plotly_specs_para_js = gerar_dados_para_js(workout_df_processed, rotinas_exercicios_js_map)

# Serializar para JSON
js_plotly_data_var = f"const graficosPlotlySpecs = {json.dumps(graficos_plotly_specs_para_js, cls=PlotlyJSONEncoder)};"
js_rotinas_map_var = f"const rotinasExerciciosMap = {json.dumps(rotinas_exercicios_js_map)};"

print(f"Número de especificações de gráfico geradas: {len(graficos_plotly_specs_para_js)}")
if graficos_plotly_specs_para_js:
   print(f"Exemplo de chave JS: {list(graficos_plotly_specs_para_js.keys())[0]}")

Número de especificações de gráfico geradas: 305
Exemplo de chave JS: Legs sábado||Deadlift (Barbell)||30||Todas as Séries


In [18]:
html_template_final = f"""
<!DOCTYPE html>
<html lang="pt-BR">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Análise de Progresso de Treinos - Hevy</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@300;400;500;700&display=swap" rel="stylesheet">
    <style>
        :root {{
            --primary-color: #007bff; /* Azul primário */
            --primary-hover-color: #0056b3;
            --secondary-color: #6c757d; /* Cinza secundário */
            --light-gray-color: #f8f9fa;
            --medium-gray-color: #e9ecef;
            --dark-gray-color: #343a40;
            --success-color: #28a745;
            --danger-color: #dc3545;
            --warning-color: #ffc107;
            --info-bg-color: #e2f3ff;
            --info-border-color: #b8daff;
            --info-text-color: #0c5464;
            --border-radius: 0.375rem; /* 6px */
            --box-shadow: 0 0.5rem 1rem rgba(0, 0, 0, 0.1);
            --input-focus-shadow: 0 0 0 0.25rem rgba(0, 123, 255, 0.25);
        }}

        * {{
            box-sizing: border-box;
            margin: 0;
            padding: 0;
        }}

        body {{ 
            font-family: 'Roboto', 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; 
            background-color: var(--light-gray-color); 
            color: var(--dark-gray-color); 
            line-height: 1.6; 
            font-size: 16px;
        }}

        header {{ 
            background-color: var(--primary-color); 
            color: white; 
            padding: 20px 25px; 
            text-align: center; 
            box-shadow: 0 2px 5px rgba(0,0,0,0.15); 
            margin-bottom: 30px;
        }}
        header h1 {{ 
            margin: 0; 
            font-size: 2em; 
            font-weight: 500;
        }}

        .container {{ 
            max-width: 1140px; /* Aumentado para layouts mais amplos */
            margin: 0 auto 30px auto; 
            padding: 30px; 
            background-color: #ffffff; 
            box-shadow: var(--box-shadow); 
            border-radius: var(--border-radius); 
        }}

        .controls-grid {{ 
            display: grid; 
            grid-template-columns: repeat(auto-fit, minmax(240px, 1fr)); /* Ajustado minmax */
            gap: 20px; 
            margin-bottom: 25px; 
        }}
        .control-group label {{ 
            font-weight: 500; 
            display: block; 
            margin-bottom: 8px; 
            font-size: 0.95em; 
            color: #495057; 
        }}
        select {{ 
            width: 100%; 
            padding: 12px 15px; 
            font-size: 1em; 
            border: 1px solid #ced4da; 
            border-radius: var(--border-radius); 
            background-color: #fff; 
            transition: border-color 0.2s ease-in-out, box-shadow 0.2s ease-in-out; 
            appearance: none; /* Remove a seta padrão do browser */
            background-image: url('data:image/svg+xml;charset=US-ASCII,%3Csvg%20width%3D%2220%22%20height%3D%2220%22%20xmlns%3D%22http%3A//www.w3.org/2000/svg%22%3E%3Cpath%20d%3D%22M5%208l5%205%205-5z%22%20fill%3D%22%23555%22/%3E%3C/svg%3E');
            background-repeat: no-repeat;
            background-position: right 15px center;
            background-size: 12px;
        }}
        select:focus {{ 
            border-color: var(--primary-color); 
            outline: 0; 
            box-shadow: var(--input-focus-shadow);
        }}

        #grafico-container {{ 
            width: 100%; 
            min-height: 500px; /* Aumentado */
            height: 520px; 
            margin-top: 25px; 
            border: 1px solid var(--medium-gray-color); 
            border-radius: var(--border-radius); 
            background-color: #fff; 
            display: block;
            position: relative;
            overflow: hidden; 
        }}
        #grafico-container p {{ 
            text-align: center;  
            color: var(--secondary-color); 
            font-size: 1.1em; 
            position: absolute;
            top: 50%;
            left: 50%;
            transform: translate(-50%, -50%);
            width: 90%;
        }}
        
        .loader-wrapper {{
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            position: absolute;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            background-color: rgba(255, 255, 255, 0.85); 
            z-index: 10; 
        }}
        .loader {{
            border: 6px solid #e9ecef; /* Cinza mais claro */
            border-top: 6px solid var(--primary-color); /* Azul primário */
            border-radius: 50%;
            width: 50px;
            height: 50px;
            animation: spin 0.8s linear infinite;
        }}
        @keyframes spin {{
            0% {{ transform: rotate(0deg); }}
            100% {{ transform: rotate(360deg); }}
        }}

        .alert {{ 
            background-color: var(--info-bg-color); 
            color: var(--info-text-color); 
            border: 1px solid var(--info-border-color); 
            padding: 18px; 
            border-radius: var(--border-radius); 
            margin-bottom: 30px; 
            font-size: 0.95em; 
        }}
        .alert strong {{ 
            color: #004085; /* Um pouco mais escuro para destaque */
            font-weight: 500;
        }}
        .alert ul {{
            margin-top: 10px;
            margin-left: 20px;
            padding-left: 0;
        }}
        .alert li {{
            margin-bottom: 5px;
        }}

        @media (max-width: 768px) {{
            .controls-grid {{ grid-template-columns: 1fr; }}
            header h1 {{ font-size: 1.6em; }}
            .container {{ margin: 15px; padding: 15px; }}
            select {{ padding: 10px 12px; background-position: right 12px center;}}
            #grafico-container {{ min-height: 400px; height: 420px; }}
        }}
    </style>
</head>
<body>
    <header><h1>Análise de Progresso de Treinos</h1></header>
    <div class="container">
        <div class="alert">
            <strong>Bem-vindo(a) à sua Análise de Progresso de Treinos!</strong>
            <ul>
                <li>Comece selecionando uma <strong>Rotina de Treino</strong> e, em seguida, um <strong>Exercício</strong> específico.</li>
                <li>O gráfico será carregado automaticamente com os dados dos últimos 30 dias para o "Top Set" (a série mais pesada até a falha) ou a primeira série de trabalho disponível.</li>
                <li>Utilize os seletores de <strong>Período</strong> e <strong>Tipo de Série</strong> para explorar diferentes visualizações do seu progresso. O gráfico se atualizará automaticamente.</li>
                <li>Passe o mouse (ou clique no caso de celulares) sobre os <strong>pontos do gráfico</strong> para ver detalhes como data, peso e repetições.</li>
                <li><strong>Dica:</strong> Para uma melhor visualização em celulares, experimente usar o aparelho na horizontal (modo paisagem) ou ativar a opção "Versão para computador" no seu navegador.</li>
            </ul>
        </div>

        <div class="controls-grid">
            <div class="control-group">
                <label for="rotina">Rotina de Treino:</label>
                <select id="rotina" onchange="rotinaSelecionada()">
                    <option value="">-- Selecione uma Rotina --</option>
                </select>
            </div>
            <div class="control-group">
                <label for="exercicio">Exercício:</label>
                <select id="exercicio" onchange="exercicioSelecionado()">
                    <option value="">-- Selecione um Exercício --</option>
                </select>
            </div>
            <div class="control-group">
                <label for="periodo">Período de Análise:</label>
                <select id="periodo" onchange="periodoOuSerieTipoAlterado(event)">
                    <option value="30" selected>Últimos 30 dias</option>
                    <option value="60">Últimos 60 dias</option>
                    <option value="90">Últimos 90 dias</option>
                </select>
            </div>
            <div class="control-group">
                <label for="serie_tipo">Visualizar Série(s):</label>
                <select id="serie_tipo" onchange="periodoOuSerieTipoAlterado(event)">
                    <option value="">-- Selecione o Tipo de Série --</option>
                </select>
            </div>
        </div>
        <div id="grafico-container"><p>Selecione uma rotina e um exercício para iniciar a análise.</p></div>
    </div>

    <script>
        {js_plotly_data_var}
        {js_rotinas_map_var}

        const rotinaSelect = document.getElementById('rotina');
        const exercicioSelect = document.getElementById('exercicio');
        const periodoSelect = document.getElementById('periodo');
        const serieTipoSelect = document.getElementById('serie_tipo');
        const graficoContainer = document.getElementById('grafico-container');

        function popularRotinas() {{
            for (const rotinaNome in rotinasExerciciosMap) {{
                if (rotinasExerciciosMap.hasOwnProperty(rotinaNome)) {{ 
                    const option = document.createElement('option');
                    option.value = rotinaNome;
                    option.text = rotinaNome;
                    rotinaSelect.appendChild(option);
                }}
            }}
        }}

        function rotinaSelecionada() {{
            const rotinaNome = rotinaSelect.value;
            exercicioSelect.innerHTML = '<option value=\"\">-- Selecione um Exercício --</option>'; 
            serieTipoSelect.innerHTML = '<option value=\"\">-- Selecione o Tipo de Série --</option>'; 
            graficoContainer.innerHTML = '<p>Selecione uma rotina e um exercício para iniciar a análise.</p>';

            if (rotinaNome && rotinasExerciciosMap[rotinaNome]) {{
                rotinasExerciciosMap[rotinaNome].forEach(exercicioNome => {{
                    const option = document.createElement('option');
                    option.value = exercicioNome;
                    option.text = exercicioNome;
                    exercicioSelect.appendChild(option);
                }});
            }}
             // Não chama exercicioSelecionado() aqui, espera a ação do usuário no dropdown de exercício
        }}

        function exercicioSelecionado() {{
            const rotinaNome = rotinaSelect.value;
            const exercicioNome = exercicioSelect.value;
            
            const currentSerieTipo = serieTipoSelect.value; 
            serieTipoSelect.innerHTML = '<option value=\"\">-- Selecione o Tipo de Série --</option>';
            
            if (!rotinaNome || !exercicioNome) {{ // Se o exercício for desmarcado
                graficoContainer.innerHTML = '<p>Selecione uma rotina e um exercício para iniciar a análise.</p>';
                return;
            }}

            const seriesSet = new Set();
            const periodoAtualParaPopularSeries = periodoSelect.value || '30'; 

            for (const chave in graficosPlotlySpecs) {{
                if (graficosPlotlySpecs.hasOwnProperty(chave)) {{
                    const partes = chave.split('||');
                    if (partes[0] === rotinaNome && partes[1] === exercicioNome && partes[2] === periodoAtualParaPopularSeries) {{
                        seriesSet.add(partes[3]);
                    }}
                }}
            }}
            
            if (seriesSet.size === 0 && periodoAtualParaPopularSeries !== '30') {{
                 for (const chave in graficosPlotlySpecs) {{
                     if (graficosPlotlySpecs.hasOwnProperty(chave)) {{
                        const partes = chave.split('||');
                        if (partes[0] === rotinaNome && partes[1] === exercicioNome && partes[2] === '30') {{
                            seriesSet.add(partes[3]);
                        }}
                    }}
                }}
            }}
            if (seriesSet.size === 0) {{ 
                for (const p of ['30', '60', '90']) {{
                    for (const chave in graficosPlotlySpecs) {{
                        if (graficosPlotlySpecs.hasOwnProperty(chave)) {{
                            const partes = chave.split('||');
                            if (partes[0] === rotinaNome && partes[1] === exercicioNome && partes[2] === p) {{
                                seriesSet.add(partes[3]);
                            }}
                        }}
                    }}
                    if (seriesSet.size > 0) break;
                }}
            }}

            const seriesOrder = ["Top Set (Failure)", "Todas as Séries"];
            const sortedSeries = Array.from(seriesSet).sort((a, b) => {{
                let aIdx = seriesOrder.indexOf(a);
                let bIdx = seriesOrder.indexOf(b);
                if (a.startsWith("Work Set")) aIdx = seriesOrder.length + parseInt(a.substring(9), 10); 
                if (b.startsWith("Work Set")) bIdx = seriesOrder.length + parseInt(b.substring(9), 10);
                aIdx = (aIdx === -1 || isNaN(aIdx)) ? Infinity : aIdx;
                bIdx = (bIdx === -1 || isNaN(bIdx)) ? Infinity : bIdx;
                return aIdx - bIdx;
            }});

            let restaurouSerie = false;
            if (sortedSeries.length > 0) {{
                sortedSeries.forEach(serieNome => {{
                    const option = document.createElement('option');
                    option.value = serieNome;
                    option.text = serieNome;
                    serieTipoSelect.appendChild(option);
                    if (serieNome === currentSerieTipo) {{
                        serieTipoSelect.value = currentSerieTipo;
                        restaurouSerie = true;
                    }}
                }});
            
                if (!restaurouSerie) {{ // Se a série anterior não existe mais para este período/exercício
                    if (seriesSet.has("Top Set (Failure)")) {{
                        serieTipoSelect.value = "Top Set (Failure)";
                    }} else if (seriesSet.has("Todas as Séries")) {{
                        serieTipoSelect.value = "Todas as Séries";
                    }} else {{
                        serieTipoSelect.value = sortedSeries[0]; // Primeira disponível da lista ordenada
                    }}
                }}
            }} else {{
                serieTipoSelect.innerHTML = '<option value=\"\">Nenhuma série aplicável</option>';
            }}
            
            carregarGrafico(); 
        }}
        
        function periodoOuSerieTipoAlterado(event) {{ 
            if (event && event.target && event.target.id === 'periodo') {{
                 exercicioSelecionado(); // Repopula séries e recarrega o gráfico
                 return; 
            }}
            // Se apenas o tipo de série mudou, e já temos rotina/exercício, recarrega o gráfico.
            const rotina = rotinaSelect.value;
            const exercicio = exercicioSelect.value;
            if (rotina && exercicio) {{ 
                carregarGrafico();
            }}
        }}

        function carregarGrafico() {{
            const rotina = rotinaSelect.value;
            const exercicio = exercicioSelect.value;
            const periodo = periodoSelect.value;
            const serieTipo = serieTipoSelect.value;

            graficoContainer.innerHTML = ''; 

            if (!rotina || !exercicio || !periodo || !serieTipo || serieTipo === "" || serieTipoSelect.options[0].text === "Nenhuma série aplicável") {{
                let msg = "<p>";
                if (!rotina || !exercicio) {{
                     msg += "Selecione uma rotina e um exercício.";
                }} else if (!serieTipo || serieTipo === "" || (serieTipoSelect.options.length > 0 && serieTipoSelect.options[0].text === "Nenhuma série aplicável" && serieTipoSelect.value === "")) {{ 
                     msg += "Nenhum tipo de série aplicável ou selecione um tipo de série válido.";
                }} else {{ 
                     msg += "Por favor, selecione todas as opções válidas.";
                }}
                msg += "</p>";
                graficoContainer.innerHTML = msg;
                return;
            }}

            const chave = `${{rotina}}||${{exercicio}}||${{periodo}}||${{serieTipo}}`;
            
            const loaderWrapper = document.createElement('div');
            loaderWrapper.className = 'loader-wrapper';
            const loaderDiv = document.createElement('div');
            loaderDiv.className = 'loader';
            const loadingText = document.createElement('p');
            loadingText.textContent = 'Carregando gráfico...';
            
            loaderWrapper.appendChild(loaderDiv);
            loaderWrapper.appendChild(loadingText);
            graficoContainer.appendChild(loaderWrapper);

            setTimeout(() => {{ 
                graficoContainer.innerHTML = ''; 
                if (graficosPlotlySpecs && graficosPlotlySpecs[chave]) {{
                    const graficoSpec = graficosPlotlySpecs[chave];
                    Plotly.newPlot(graficoContainer, graficoSpec.data, graficoSpec.layout, {{responsive: true, displaylogo: false}});
                }} else {{
                    graficoContainer.innerHTML = "<p>Dados não disponíveis para esta combinação. <br>Verifique se há registros para o período e tipo de série selecionados ou tente outra combinação.</p>";
                }}
            }}, 50);
        }}

        document.addEventListener('DOMContentLoaded', () => {{
            popularRotinas();
        }});
    </script>
</body>
</html>
"""

with open("visualizacao_treino_profissional.html", "w", encoding="utf-8") as f:
    f.write(html_template_final)

print("Arquivo HTML 'visualizacao_treino_profissional.html' gerado.")

Arquivo HTML 'visualizacao_treino_profissional.html' gerado.
